## Calculating similarity score using Jaro Winkler distance.

In [1]:
!pip install strsimpy

In [2]:
# importing all necessary libraries
import pandas as pd
import numpy as np
import json, collections
from pandas.io.json import json_normalize
import os

In [3]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re
pd.set_option('max_colwidth', 800)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df1 = pd.read_csv('/content/Chatgptcases.csv', encoding='ISO-8859-1')
df = df1.head(20)
df

,Case/Question,Clinical Pharmacist answer,Chatgpt answer
0,Which antihypertensive medications are more effective in African Americans?,"Although all classes of antihypertensive medication have been shown to be comparable in lowering blood pressure, thiazide-type diuretics, and calcium channel blockers (CCBs) are recommended as first-line treatment for uncomplicated hypertension in blacks. ACEI/ARBs are recommended as first-line agents in blacks only with comorbid CKD (moderate recommendation); the two most important trials cited in the introduction of race in the algorithmic treatment of HTN are Antihypertensive and Lipid-Lowering Treatment to Prevent Heart Attack Trial (ALLHAT) and African American Study of Kidney and Hypertension (AASK).","I'm not a doctor, but I can try to provide you with some general information on antihypertensive medications. According to some research, certain antihypertensive medications may be more effective in African Americans compared to other racial or ethnic groups. It's important to note, though, that treatment should always be individualized, and you should consult a healthcare professional for specific advice.\n\nTwo classes of antihypertensive medications that are often found to be more effective in African Americans are:\n\n1. **Calcium channel blockers (CCBs):** These medications work by relaxing the blood vessels and reducing the force with which the heart pumps blood. Some common calcium channel blockers include amlodipine (Norvasc), diltiazem (Cardizem, Tiazac), and nifedipine (Adal..."
1,What type of antihypertensive drugs are shown to reduce BP and reduce the progression of diabetic neuropathy?,"ACE Inhibitors (Angiotensin-Converting Enzyme Inhibitors) and ARBs (Angiotensin II Receptor Blockers) are used as first-line treatment can help lower blood pressure and may have a protective effect on the kidneys and nerves, potentially slowing down the progression of diabetic neuropathy. ACEIs can be used alone or combined with a thiazide-type diuretic or other antihypertensive drug.","Antihypertensive drugs are used to lower blood pressure and can help slow the progression of diabetic neuropathy in patients with diabetes. Some of the main classes of antihypertensive drugs that have been shown to be effective in this regard include:\n\n1. **Angiotensin-converting enzyme (ACE) inhibitors:** These drugs block the action of angiotensin-converting enzyme, which is responsible for the production of angiotensin II, a potent vasoconstrictor. By reducing the production of angiotensin II, ACE inhibitors help dilate blood vessels and lower blood pressure. Examples of ACE inhibitors include lisinopril, enalapril, and captopril. ACE inhibitors have been shown to reduce the progression of diabetic nephropathy in patients with diabetes.\n\n1. **Angiotensin II receptor blockers (AR..."
2,What are the side effects of amlodipine?,"The following adverse drug reactions and incidences are derived from product labelling unless otherwise specified.\n\n>10%: Cardiovascular: Peripheral edema (2% to 11%, dose related; females: 15%; males: 6%) \n\n1% to 10%:Cardiovascular: Flushing (?3%, dose related, more frequent in females), palpitations (?5%, dose related, more frequent in females)\n\nDermatologic: Pruritus (?2%), skin rash (?2%)\n\nGastrointestinal: Abdominal pain (2%), nausea (3%)\n\nGenitourinary: Male sexual disorder (?2%)\n\nNervous system: Asthenia (?2%), dizziness (doses ?5 mg/day: 3%), drowsiness (1% to 2%), fatigue (5%)\n\nNeuromuscular & skeletal: Muscle cramps (?2%)\n\nRespiratory: Dyspnea (?2%)\n\n<1%:Cardiovascular: Peripheral ischemia, sinus tachycardia, syncope, vasculitis\nDermatologic: Diaphoresis, e...","Amlodipine is a calcium channel blocker used to treat high blood pressure (hypertension) and chest pain (angina). As with any medication, it can cause side effects. However, not everyone who takes amlodipine will experience these side effects, and they are generally mild and manageable. Some of t

In [5]:
df.columns

Index(['Case/Question', 'Clinical Pharmacist answer', 'Chatgpt answer'], dtype='object')

In [6]:
# Assuming your DataFrame is df
selected_columns = ['Clinical Pharmacist answer', 'Chatgpt answer']  # Replace 'Column1' and 'Column2' with your actual column names

# Select only the specified columns
df2 = df[selected_columns]

# Display the resulting DataFrame
df2

,Clinical Pharmacist answer,Chatgpt answer
0,"Although all classes of antihypertensive medication have been shown to be comparable in lowering blood pressure, thiazide-type diuretics, and calcium channel blockers (CCBs) are recommended as first-line treatment for uncomplicated hypertension in blacks. ACEI/ARBs are recommended as first-line agents in blacks only with comorbid CKD (moderate recommendation); the two most important trials cited in the introduction of race in the algorithmic treatment of HTN are Antihypertensive and Lipid-Lowering Treatment to Prevent Heart Attack Trial (ALLHAT) and African American Study of Kidney and Hypertension (AASK).","I'm not a doctor, but I can try to provide you with some general information on antihypertensive medications. According to some research, certain antihypertensive medications may be more effective in African Americans compared to other racial or ethnic groups. It's important to note, though, that treatment should always be individualized, and you should consult a healthcare professional for specific advice.\n\nTwo classes of antihypertensive medications that are often found to be more effective in African Americans are:\n\n1. **Calcium channel blockers (CCBs):** These medications work by relaxing the blood vessels and reducing the force with which the heart pumps blood. Some common calcium channel blockers include amlodipine (Norvasc), diltiazem (Cardizem, Tiazac), and nifedipine (Adal..."
1,"ACE Inhibitors (Angiotensin-Converting Enzyme Inhibitors) and ARBs (Angiotensin II Receptor Blockers) are used as first-line treatment can help lower blood pressure and may have a protective effect on the kidneys and nerves, potentially slowing down the progression of diabetic neuropathy. ACEIs can be used alone or combined with a thiazide-type diuretic or other antihypertensive drug.","Antihypertensive drugs are used to lower blood pressure and can help slow the progression of diabetic neuropathy in patients with diabetes. Some of the main classes of antihypertensive drugs that have been shown to be effective in this regard include:\n\n1. **Angiotensin-converting enzyme (ACE) inhibitors:** These drugs block the action of angiotensin-converting enzyme, which is responsible for the production of angiotensin II, a potent vasoconstrictor. By reducing the production of angiotensin II, ACE inhibitors help dilate blood vessels and lower blood pressure. Examples of ACE inhibitors include lisinopril, enalapril, and captopril. ACE inhibitors have been shown to reduce the progression of diabetic nephropathy in patients with diabetes.\n\n1. **Angiotensin II receptor blockers (AR..."
2,"The following adverse drug reactions and incidences are derived from product labelling unless otherwise specified.\n\n>10%: Cardiovascular: Peripheral edema (2% to 11%, dose related; females: 15%; males: 6%) \n\n1% to 10%:Cardiovascular: Flushing (?3%, dose related, more frequent in females), palpitations (?5%, dose related, more frequent in females)\n\nDermatologic: Pruritus (?2%), skin rash (?2%)\n\nGastrointestinal: Abdominal pain (2%), nausea (3%)\n\nGenitourinary: Male sexual disorder (?2%)\n\nNervous system: Asthenia (?2%), dizziness (doses ?5 mg/day: 3%), drowsiness (1% to 2%), fatigue (5%)\n\nNeuromuscular & skeletal: Muscle cramps (?2%)\n\nRespiratory: Dyspnea (?2%)\n\n<1%:Cardiovascular: Peripheral ischemia, sinus tachycardia, syncope, vasculitis\nDermatologic: Diaphoresis, e...","Amlodipine is a calcium channel blocker used to treat high blood pressure (hypertension) and chest pain (angina). As with any medication, it can cause side effects. However, not everyone who takes amlodipine will experience these side effects, and they are generally mild and manageable. Some of the most common side effects of amlodipine include:\n\n- **Swelling (edema)**: This can occur in the feet, ankles, or lower legs. Elevating the affected area and wearing compression stockings may help alleviate the swelling.\n- **Dizziness or

In [7]:
# Cleaning Raw tweets
def clean_text(text):

    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])

    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)

    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)

    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces
    text = re.sub('\s+', ' ', text)

    return text

In [8]:
# applying that into function.
df2["Clinical_answer"] = df2['Clinical Pharmacist answer'].apply(lambda x: clean_text(x))
df2["Chatgpt_answer"] = df2['Chatgpt answer'].apply(lambda x: clean_text(x))
df2

,Clinical Pharmacist answer,Chatgpt answer,Clinical_answer,Chatgpt_answer
0,"Although all classes of antihypertensive medication have been shown to be comparable in lowering blood pressure, thiazide-type diuretics, and calcium channel blockers (CCBs) are recommended as first-line treatment for uncomplicated hypertension in blacks. ACEI/ARBs are recommended as first-line agents in blacks only with comorbid CKD (moderate recommendation); the two most important trials cited in the introduction of race in the algorithmic treatment of HTN are Antihypertensive and Lipid-Lowering Treatment to Prevent Heart Attack Trial (ALLHAT) and African American Study of Kidney and Hypertension (AASK).","I'm not a doctor, but I can try to provide you with some general information on antihypertensive medications. According to some research, certain antihypertensive medications may be more effective in African Americans compared to other racial or ethnic groups. It's important to note, though, that treatment should always be individualized, and you should consult a healthcare professional for specific advice.\n\nTwo classes of antihypertensive medications that are often found to be more effective in African Americans are:\n\n1. **Calcium channel blockers (CCBs):** These medications work by relaxing the blood vessels and reducing the force with which the heart pumps blood. Some common calcium channel blockers include amlodipine (Norvasc), diltiazem (Cardizem, Tiazac), and nifedipine (Adal...",Although all classes of antihypertensive medication have been shown to be comparable in lowering blood pressure thiazide type diuretics and calcium channel blockers are recommended as first line treatment for uncomplicated hypertension in blacks ACEI ARBs are recommended as first line agents in blacks only with comorbid CKD the two most important trials cited in the introduction of race in the algorithmic treatment of HTN are Antihypertensive and Lipid Lowering Treatment to Prevent Heart Attack Trial and African American Study of Kidney and Hypertension,I m not a doctor but I can try to provide you with some general information on antihypertensive medications According to some research certain antihypertensive medications may be more effective in African Americans compared to other racial or ethnic groups It s important to note though that treatment should always be individualized and you should consult a healthcare professional for specific advice Two classes of antihypertensive medications that are often found to be more effective in African Americans are Calcium channel blockers These medications work by relaxing the blood vessels and reducing the force with which the heart pumps blood Some common calcium channel blockers include amlodipine diltiazem and nifedipine Thiazide diuretics These medications help the kidneys remove excess...
1,"ACE Inhibitors (Angiotensin-Converting Enzyme Inhibitors) and ARBs (Angiotensin II Receptor Blockers) are used as first-line treatment can help lower blood pressure and may have a protective effect on the kidneys and nerves, potentially slowing down the progression of diabetic neuropathy. ACEIs can be used alone or combined with a thiazide-type diuretic or other antihypertensive drug.","Antihypertensive drugs are used to lower blood pressure and can help slow the progression of diabetic neuropathy in patients with diabetes. Some of the main classes of antihypertensive drugs that have been shown to be effective in this regard include:\n\n1. **Angiotensin-converting enzyme (ACE) inhibitors:** These drugs block the action of angiotensin-converting enzyme, which is responsible for the production of angiotensin II, a potent vasoconstrictor. By reducing the production of angiotensin II, ACE inhibitors help dilate blood vessels and lower blood pressure. Examples of ACE inhibitors include lisinopril, enalapril, and captopril. ACE inhibitors have been shown to reduce the progression of diabetic nephropathy in patients with diabetes.\n

In [9]:
df2.drop(["Clinical Pharmacist answer", "Chatgpt answer"], axis=1, inplace=True)
df2.head()

,Clinical_answer,Chatgpt_answer
0,Although all classes of antihypertensive medication have been shown to be comparable in lowering blood pressure thiazide type diuretics and calcium channel blockers are recommended as first line treatment for uncomplicated hypertension in blacks ACEI ARBs are recommended as first line agents in blacks only with comorbid CKD the two most important trials cited in the introduction of race in the algorithmic treatment of HTN are Antihypertensive and Lipid Lowering Treatment to Prevent Heart Attack Trial and African American Study of Kidney and Hypertension,I m not a doctor but I can try to provide you with some general information on antihypertensive medications According to some research certain antihypertensive medications may be more effective in African Americans compared to other racial or ethnic groups It s important to note though that treatment should always be individualized and you should consult a healthcare professional for specific advice Two classes of antihypertensive medications that are often found to be more effective in African Americans are Calcium channel blockers These medications work by relaxing the blood vessels and reducing the force with which the heart pumps blood Some common calcium channel blockers include amlodipine diltiazem and nifedipine Thiazide diuretics These medications help the kidneys remove excess...
1,ACE Inhibitors and ARBs are used as first line treatment can help lower blood pressure and may have a protective effect on the kidneys and nerves potentially slowing down the progression of diabetic neuropathy ACEIs can be used alone or combined with a thiazide type diuretic or other antihypertensive drug,Antihypertensive drugs are used to lower blood pressure and can help slow the progression of diabetic neuropathy in patients with diabetes Some of the main classes of antihypertensive drugs that have been shown to be effective in this regard include Angiotensin converting enzyme inhibitors These drugs block the action of angiotensin converting enzyme which is responsible for the production of angiotensin II a potent vasoconstrictor By reducing the production of angiotensin II ACE inhibitors help dilate blood vessels and lower blood pressure Examples of ACE inhibitors include lisinopril enalapril and captopril ACE inhibitors have been shown to reduce the progression of diabetic nephropathy in patients with diabetes Angiotensin II receptor blockers ARBs work by blocking the action of ang...
2,The following adverse drug reactions and incidences are derived from product labelling unless otherwise specified Cardiovascular Peripheral edema to Cardiovascular Flushing palpitations Dermatologic Pruritus skin rash Gastrointestinal Abdominal pain nausea Genitourinary Male sexual disorder Nervous system Asthenia dizziness drowsiness fatigue Neuromuscular skeletal Muscle cramps Respiratory Dyspnea Cardiovascular Peripheral ischemia sinus tachycardia syncope vasculitis Dermatologic Diaphoresis erythema multiforme Endocrine metabolic Hot flash hyperglycemia weight gain weight loss Gastrointestinal Anorexia constipation dysphagia flatulence gingival hyperplasia pancreatitis vomiting xerostomia Genitourinary Difficulty in micturition female sexual disorder nocturia urinary frequency Hemat...,Amlodipine is a calcium channel blocker used to treat high blood pressure and chest pain As with any medication it can cause side effects However not everyone who takes amlodipine will experience these side effects and they are generally mild and manageable Some of the most common side effects of amlodipine include Swelling This can occur in the feet ankles or lower legs Elevating the affected area and wearing compression stockings may help alleviate the swelling Dizziness or lightheadedness This may be more common when you first start taking amlodipine or when your dose is increased Standing up slowly and avoiding sudden changes in position can help prevent dizziness Headache Over the 

In [10]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()
def clean_text(review):
  review = review.lower()
  review = review.split()
  review = [wnl.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  return review

In [12]:
df2["Clinical_answer"] = df2['Clinical_answer'].apply(clean_text)
df2["Chatgpt_answer"] = df2['Chatgpt_answer'].apply(clean_text)
df2

,Clinical_answer,Chatgpt_answer
0,although class antihypertensive medication shown comparable lowering blood pressure thiazide type diuretic calcium channel blocker recommended first line treatment uncomplicated hypertension black acei arb recommended first line agent black comorbid ckd two important trial cited introduction race algorithmic treatment htn antihypertensive lipid lowering treatment prevent heart attack trial african american study kidney hypertension,doctor try provide general information antihypertensive medication according research certain antihypertensive medication may effective african american compared racial ethnic group important note though treatment always individualized consult healthcare professional specific advice two class antihypertensive medication often found effective african american calcium channel blocker medication work relaxing blood vessel reducing force heart pump blood common calcium channel blocker include amlodipine diltiazem nifedipine thiazide diuretic medication help kidney remove excess sodium water body help lower blood pressure common thiazide diuretic include hydrochlorothiazide chlorthalidone indapamide worth mentioning american college cardiology american heart association hypertension guideli...
1,ace inhibitor arb used first line treatment help lower blood pressure may protective effect kidney nerve potentially slowing progression diabetic neuropathy aceis used alone combined thiazide type diuretic antihypertensive drug,antihypertensive drug used lower blood pressure help slow progression diabetic neuropathy patient diabetes main class antihypertensive drug shown effective regard include angiotensin converting enzyme inhibitor drug block action angiotensin converting enzyme responsible production angiotensin ii potent vasoconstrictor reducing production angiotensin ii ace inhibitor help dilate blood vessel lower blood pressure example ace inhibitor include lisinopril enalapril captopril ace inhibitor shown reduce progression diabetic nephropathy patient diabetes angiotensin ii receptor blocker arb work blocking action angiotensin ii receptor help dilate blood vessel lower blood pressure example arb include losartan valsartan candesartan like ace inhibitor arb shown reduce progression diabetic nephropa...
2,following adverse drug reaction incidence derived product labelling unless otherwise specified cardiovascular peripheral edema cardiovascular flushing palpitation dermatologic pruritus skin rash gastrointestinal abdominal pain nausea genitourinary male sexual disorder nervous system asthenia dizziness drowsiness fatigue neuromuscular skeletal muscle cramp respiratory dyspnea cardiovascular peripheral ischemia sinus tachycardia syncope vasculitis dermatologic diaphoresis erythema multiforme endocrine metabolic hot flash hyperglycemia weight gain weight loss gastrointestinal anorexia constipation dysphagia flatulence gingival hyperplasia pancreatitis vomiting xerostomia genitourinary difficulty micturition female sexual disorder nocturia urinary frequency hematologic oncologic leukopenia...,amlodipine calcium channel blocker used treat high blood pressure chest pain medication cause side effect however everyone take amlodipine experience side effect generally mild manageable common side effect amlodipine include swelling occur foot ankle lower leg elevating affected area wearing compression stocking may help alleviate swelling dizziness lightheadedness may common first start taking amlodipine dose increased standing slowly avoiding sudden change position help prevent dizziness headache counter pain reliever acetaminophen ibuprofen help alleviate headache fatigue may improve time body adjusts medication fatigue persists worsens contact healthcare provider palpitation irregular heartbeat symptom persistent severe contact healthcare provider nausea taking amlodipine food may...
3,lifestyle modification effectively lower blood pressure increased physical activity weight loss 

In [13]:
from strsimpy.cosine import Cosine
cosine = Cosine(2)
df2["p0"] = df2["Clinical_answer"].apply(lambda s: cosine.get_profile(s))
df2["p1"] = df2["Chatgpt_answer"].apply(lambda s: cosine.get_profile(s))
df2["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df2["p0"],df2["p1"])]

In [14]:
df2.drop(["p0", "p1"], axis=1, inplace=True)
df2.head()

,Clinical_answer,Chatgpt_answer,cosine_sim
0,although class antihypertensive medication shown comparable lowering blood pressure thiazide type diuretic calcium channel blocker recommended first line treatment uncomplicated hypertension black acei arb recommended first line agent black comorbid ckd two important trial cited introduction race algorithmic treatment htn antihypertensive lipid lowering treatment prevent heart attack trial african american study kidney hypertension,doctor try provide general information antihypertensive medication according research certain antihypertensive medication may effective african american compared racial ethnic group important note though treatment always individualized consult healthcare professional specific advice two class antihypertensive medication often found effective african american calcium channel blocker medication work relaxing blood vessel reducing force heart pump blood common calcium channel blocker include amlodipine diltiazem nifedipine thiazide diuretic medication help kidney remove excess sodium water body help lower blood pressure common thiazide diuretic include hydrochlorothiazide chlorthalidone indapamide worth mentioning american college cardiology american heart association hypertension guideli...,0.849266
1,ace inhibitor arb used first line treatment help lower blood pressure may protective effect kidney nerve potentially slowing progression diabetic neuropathy aceis used alone combined thiazide type diuretic antihypertensive drug,antihypertensive drug used lower blood pressure help slow progression diabetic neuropathy patient diabetes main class antihypertensive drug shown effective regard include angiotensin converting enzyme inhibitor drug block action angiotensin converting enzyme responsible production angiotensin ii potent vasoconstrictor reducing production angiotensin ii ace inhibitor help dilate blood vessel lower blood pressure example ace inhibitor include lisinopril enalapril captopril ace inhibitor shown reduce progression diabetic nephropathy patient diabetes angiotensin ii receptor blocker arb work blocking action angiotensin ii receptor help dilate blood vessel lower blood pressure example arb include losartan valsartan candesartan like ace inhibitor arb shown reduce progression diabetic nephropa...,0.814322
2,following adverse drug reaction incidence derived product labelling unless otherwise specified cardiovascular peripheral edema cardiovascular flushing palpitation dermatologic pruritus skin rash gastrointestinal abdominal pain nausea genitourinary male sexual disorder nervous system asthenia dizziness drowsiness fatigue neuromuscular skeletal muscle cramp respiratory dyspnea cardiovascular peripheral ischemia sinus tachycardia syncope vasculitis dermatologic diaphoresis erythema multiforme endocrine metabolic hot flash hyperglycemia weight gain weight loss gastrointestinal anorexia constipation dysphagia flatulence gingival hyperplasia pancreatitis vomiting xerostomia genitourinary difficulty micturition female sexual disorder nocturia urinary frequency hematologic oncologic leukopenia...,amlodipine calcium channel blocker used treat high blood pressure chest pain medication cause side effect however everyone take amlodipine experience side effect generally mild manageable common side effect amlodipine include swelling occur foot ankle lower leg elevating affected area wearing compression stocking may help alleviate swelling dizziness lightheadedness may common first start taking amlodipine dose increased standing slowly avoiding sudden change position help prevent dizziness headache counter pain reliever acetaminophen ibuprofen help alleviate headache fatigue may improve time body adjusts medication fatigue persists worsens contact healthcare provider palpitation irregular heartbeat symptom persistent severe contact healthcare provider nausea taking amlodipine food may...,0.759643
3,lifestyle modification effectively lower blood pressure in

In [15]:
df2

,Clinical_answer,Chatgpt_answer,cosine_sim
0,although class antihypertensive medication shown comparable lowering blood pressure thiazide type diuretic calcium channel blocker recommended first line treatment uncomplicated hypertension black acei arb recommended first line agent black comorbid ckd two important trial cited introduction race algorithmic treatment htn antihypertensive lipid lowering treatment prevent heart attack trial african american study kidney hypertension,doctor try provide general information antihypertensive medication according research certain antihypertensive medication may effective african american compared racial ethnic group important note though treatment always individualized consult healthcare professional specific advice two class antihypertensive medication often found effective african american calcium channel blocker medication work relaxing blood vessel reducing force heart pump blood common calcium channel blocker include amlodipine diltiazem nifedipine thiazide diuretic medication help kidney remove excess sodium water body help lower blood pressure common thiazide diuretic include hydrochlorothiazide chlorthalidone indapamide worth mentioning american college cardiology american heart association hypertension guideli...,0.849266
1,ace inhibitor arb used first line treatment help lower blood pressure may protective effect kidney nerve potentially slowing progression diabetic neuropathy aceis used alone combined thiazide type diuretic antihypertensive drug,antihypertensive drug used lower blood pressure help slow progression diabetic neuropathy patient diabetes main class antihypertensive drug shown effective regard include angiotensin converting enzyme inhibitor drug block action angiotensin converting enzyme responsible production angiotensin ii potent vasoconstrictor reducing production angiotensin ii ace inhibitor help dilate blood vessel lower blood pressure example ace inhibitor include lisinopril enalapril captopril ace inhibitor shown reduce progression diabetic nephropathy patient diabetes angiotensin ii receptor blocker arb work blocking action angiotensin ii receptor help dilate blood vessel lower blood pressure example arb include losartan valsartan candesartan like ace inhibitor arb shown reduce progression diabetic nephropa...,0.814322
2,following adverse drug reaction incidence derived product labelling unless otherwise specified cardiovascular peripheral edema cardiovascular flushing palpitation dermatologic pruritus skin rash gastrointestinal abdominal pain nausea genitourinary male sexual disorder nervous system asthenia dizziness drowsiness fatigue neuromuscular skeletal muscle cramp respiratory dyspnea cardiovascular peripheral ischemia sinus tachycardia syncope vasculitis dermatologic diaphoresis erythema multiforme endocrine metabolic hot flash hyperglycemia weight gain weight loss gastrointestinal anorexia constipation dysphagia flatulence gingival hyperplasia pancreatitis vomiting xerostomia genitourinary difficulty micturition female sexual disorder nocturia urinary frequency hematologic oncologic leukopenia...,amlodipine calcium channel blocker used treat high blood pressure chest pain medication cause side effect however everyone take amlodipine experience side effect generally mild manageable common side effect amlodipine include swelling occur foot ankle lower leg elevating affected area wearing compression stocking may help alleviate swelling dizziness lightheadedness may common first start taking amlodipine dose increased standing slowly avoiding sudden change position help prevent dizziness headache counter pain reliever acetaminophen ibuprofen help alleviate headache fatigue may improve time body adjusts medication fatigue persists worsens contact healthcare provider palpitation irregular heartbeat symptom persistent severe contact healthcare provider nausea taking amlodipine food may...,0.759643
3,lifestyle modification effectively lower blood pressure in